In [1]:
# Datos
pr_era5_1 = readRDS('../../data/npd_s_w/pr_npdw_era5_1.rds')
pr_seas5_1 = readRDS('../../data/npd_s_w/pr_npdw_seas5_1.rds')
pr_seas5_1_bc = readRDS('../../data/npd_s_w/pr_npdw_seas5_1_bc.rds')

In [2]:
library(dplyr)
library(abind)
library(loadeR)
library(gridExtra)
library(loadeR.2nc)
library(visualizeR)
library(transformeR)
library(RColorBrewer)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: transformeR




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.



In [3]:
## Máscara de tierra de ERA5 (es una variable más del propio reanális):
## Valores continuos entre 0 (no hay nada de tierra en ese gridbox) y 1 (todo el gridbox es tierra)
mask = loadGridData("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5/lsm/lsm_era5.nc", var = "lsm") %>% suppressMessages %>% suppressWarnings

## Binarizo la máscara: Considero que todos los gridboxes con un valor por encima (debajo) de 0.5 son de tierra (mar)
mask.bin = binaryGrid(mask, condition = "GE", threshold = 0.5, values = c(NA, 1))

## Hago el upscaling como hice con los datos de ERA5 a la resolución de 1º del modelo
mask_upscaled = interpGrid(mask.bin,
                           new.coordinates = getGrid(pr_seas5_1),
                           method = "bilinear") %>% suppressMessages %>% suppressWarnings

## Apoyándome en la máscara binaria, me quedo únicamente con los datos en tierra y descarto el mar
mask.bin.spain = subsetGrid(mask_upscaled, lonLim = c(-10, 5), latLim = c(35, 44))
mask.bin.spain$Data = aperm(replicate(getShape(pr_seas5_1)["time"], mask.bin.spain$Data, simplify = "array"), c(3, 1, 2))
attributes(mask.bin.spain$Data)$dimensions = c("time", "lat", "lon")

## Máscara para el moodelo
n.members = getShape(pr_seas5_1)["member"]
mask.data = mask.bin.spain$Data
mask.4d = array(NA, dim = c(n.members, dim(mask.data)))  # member x time x lat x lon
for (m in 1:n.members) {
    mask.4d[m,,,] = mask.data
}

mask.model = mask.bin.spain  # copia de la estructura
mask.model$Data = mask.4d
attributes(mask.model$Data)$dimensions = c("member", "time", "lat", "lon")

In [4]:
pr_seas5_1 = gridArithmetics(pr_seas5_1, mask.model, operator = '*')

In [5]:
# 1. BINARIZACIÓN (Definir umbral de día lluvioso)
umbral = 1.0

bin_era5 = binaryGrid(pr_era5_1, condition = "GE", threshold = umbral)
bin_seas5 = binaryGrid(pr_seas5_1, condition = "GE", threshold = umbral)
bin_seas5_bc = binaryGrid(pr_seas5_1_bc, condition = "GE", threshold = umbral)

# 2. CÁLCULO DE FRECUENCIA (CLIMATOLOGÍA)
freq_era5 = climatology(bin_era5)
freq_seas5 = climatology(bin_seas5, by.member = FALSE)
freq_seas5_bc = climatology(bin_seas5_bc, by.member = FALSE)


# 3. CONVERSIÓN A PORCENTAJE
pct_era5 = gridArithmetics(freq_era5, 100, operator = "*")
pct_seas5 = gridArithmetics(freq_seas5, 100, operator = "*")
pct_seas5_bc = gridArithmetics(freq_seas5_bc, 100, operator = "*")

# Asignamos nombres para el gráfico
pct_era5$Variable$varName = "ERA5 (Obs)"
pct_seas5$Variable$varName = "SEAS5 (Raw)"
pct_seas5_bc$Variable$varName = "SEAS5 (BC)"

# Unimos los 3 grids
multigrid_freq = makeMultiGrid(pct_seas5_bc, pct_seas5, pct_era5, skip.temporal.check = TRUE)

# 4. VISUALIZACIÓN (MAPA ESPACIAL)
freq_dias_lluvia = spatialPlot(multigrid_freq,
                               backdrop.theme = "countries",
                               names.attr = c("SEAS5 (bc)", "SEAS5 (raw)", "ERA5-Land"),
                               col.regions = colorRampPalette(c("saddlebrown", "khaki", "forestgreen")),
                               at = seq(0, 60, 1),
                               set.min = 0, set.max = 60,
                               main = "Frecuencia de dias lluviosos (%)",
                               layout = c(1, 3))

[2025-12-18 13:03:56.407585] - Computing climatology...

[2025-12-18 13:03:56.424359] - Done.

[2025-12-18 13:03:56.596496] - Aggregating members...

[2025-12-18 13:04:05.017141] - Done.

[2025-12-18 13:04:05.021113] - Computing climatology...

[2025-12-18 13:04:05.039913] - Done.

[2025-12-18 13:04:05.207721] - Aggregating members...

[2025-12-18 13:04:13.773704] - Done.

[2025-12-18 13:04:13.777458] - Computing climatology...

[2025-12-18 13:04:13.7935] - Done.



In [6]:
# 1. ENMASCARAR DÍAS SECOS (Convertir < 1mm a NA)
umbral <- 1.0

# Función auxiliar para convertir valores secos a NA
mask_dry_days = function(grid, threshold) {
    grid_wet = grid
    # Asignamos NA a los datos menores al umbral
    grid_wet$Data[grid_wet$Data < threshold] = NA
    return(grid_wet)
}

# Aplicamos la máscara (ahora los días secos son NA)
pr_era5_wet = mask_dry_days(pr_era5_1, umbral)
pr_seas5_wet = mask_dry_days(pr_seas5_1, umbral)
pr_seas5_bc_wet = mask_dry_days(pr_seas5_1_bc, umbral)

# 2. CÁLCULO DE INTENSIDAD (SDII)
int_era5 = climatology(pr_era5_wet, clim.fun = list(FUN = "mean", na.rm = TRUE))
int_seas5 = climatology(pr_seas5_wet, clim.fun = list(FUN = "mean", na.rm = TRUE), by.member = FALSE)
int_seas5_bc = climatology(pr_seas5_bc_wet, clim.fun = list(FUN = "mean", na.rm = TRUE), by.member = FALSE)

# 3. PREPARACIÓN PARA GRAFICAR
multigrid_int = makeMultiGrid(int_seas5_bc, int_seas5, int_era5, skip.temporal.check = TRUE)

# 4. VISUALIZACIÓN
intensidad_lluvia = spatialPlot(multigrid_int,
                                backdrop.theme = "countries",
                                names.attr = c("SEAS5 (bc)", "SEAS5 (raw)", "ERA5-Land"),
                                col.regions = colorRampPalette(c("white", "skyblue", "darkblue")),
                                at = seq(0, 12, 0.5),
                                set.min = 0,
                                set.max = 12,
                                main = "Intensidad media en dias lluviosos (mm)",
                                layout = c(1, 3))

[2025-12-18 13:04:14.601232] - Computing climatology...

[2025-12-18 13:04:14.618311] - Done.

[2025-12-18 13:04:14.781601] - Aggregating members...

[2025-12-18 13:04:22.202444] - Done.

[2025-12-18 13:04:22.206232] - Computing climatology...

[2025-12-18 13:04:22.222248] - Done.

[2025-12-18 13:04:22.382173] - Aggregating members...

[2025-12-18 13:04:29.683713] - Done.

[2025-12-18 13:04:29.688018] - Computing climatology...

[2025-12-18 13:04:29.70659] - Done.



In [7]:
png("metricas_lluvia_npd_w.png", width = 2000, height = 1000, res = 150)
grid.arrange(freq_dias_lluvia, intensidad_lluvia, ncol = 2)
dev.off()

pdf 
  2